# Time-Terrain-Behavior (TTB) Analysis

This notebook implements the **Time-Terrain-Behavior (TTB)** framework to detect anomalous entities based on the diversity of their activity.

### The 3 Dimensions
- **Terrain (X)**: Diversity of data sources/tools observing the entity.
- **Time (Y)**: Diversity of time contexts (e.g., Weekday Morning vs. Weekend Night).
- **Behavior (Z)**: Diversity of action types (e.g., Login, Execute, Download).

**Hypothesis**: Malicious actors often exhibit higher diversity across these dimensions than normal users.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Mock Data Loader - In production, replace this with Scanner query results
def load_data():
    # Simulating logs from multiple sources
    data = [
        # Normal User: 9-5, mostly Email/File access, limited sources
        {"timestamp": "2023-10-23 09:00:00", "user": "alice", "source": "okta", "action": "login"},
        {"timestamp": "2023-10-23 09:05:00", "user": "alice", "source": "gmail", "action": "read"},
        {"timestamp": "2023-10-23 10:00:00", "user": "alice", "source": "drive", "action": "view"},
        {"timestamp": "2023-10-23 17:00:00", "user": "alice", "source": "okta", "action": "logout"},
        
        # Malicious User (Bob): Odd hours, many sources, many actions
        {"timestamp": "2023-10-23 02:00:00", "user": "bob", "source": "vpn", "action": "connect"},  # Night
        {"timestamp": "2023-10-23 02:05:00", "user": "bob", "source": "ssh", "action": "login"},
        {"timestamp": "2023-10-23 02:10:00", "user": "bob", "source": "aws", "action": "DescribeInstances"},
        {"timestamp": "2023-10-23 03:00:00", "user": "bob", "source": "github", "action": "clone"},
        {"timestamp": "2023-10-23 03:30:00", "user": "bob", "source": "gcp", "action": "setIamPolicy"},
        {"timestamp": "2023-10-23 09:00:00", "user": "bob", "source": "okta", "action": "login"},   # Morning too
        {"timestamp": "2023-10-24 20:00:00", "user": "bob", "source": "drive", "action": "download"} # Weekend/Evening
    ]
    return pd.DataFrame(data)

df = load_data()
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()

## 1. Time Dimension (Y-Axis)
Categorize activity into business contexts (e.g., Weekday Morning, Weekend Night).

In [ ]:
def get_time_context(dt):
    hour = dt.hour
    is_weekend = dt.weekday() >= 5
    
    if is_weekend:
        if 0 <= hour < 6: return "Weekend_Night"
        if 6 <= hour < 18: return "Weekend_Day"
        return "Weekend_Evening"
    else:
        if 0 <= hour < 6: return "Weekday_Night"
        if 6 <= hour < 9: return "Weekday_Early"
        if 9 <= hour < 17: return "Weekday_Core"
        if 17 <= hour < 22: return "Weekday_Evening"
        return "Weekday_Late"

df['time_context'] = df['timestamp'].apply(get_time_context)
df[['timestamp', 'user', 'time_context']].head()

## 2. Terrain Dimension (X-Axis)
Map data sources to terrain categories.

In [ ]:
def get_terrain_category(source):
    mapping = {
        "okta": "Identity",
        "vpn": "Network",
        "ssh": "Endpoint",
        "aws": "Cloud_Infra",
        "gcp": "Cloud_Infra",
        "github": "Code_Repo",
        "gmail": "SaaS",
        "drive": "SaaS"
    }
    return mapping.get(source, "Other")

df['terrain_category'] = df['source'].apply(get_terrain_category)
df[['user', 'source', 'terrain_category']].head()

## 3. Behavior Dimension (Z-Axis)
Map actions to behavior categories.

In [ ]:
def get_behavior_category(action):
    action = action.lower()
    if "login" in action or "connect" in action: return "Access"
    if "read" in action or "view" in action or "describe" in action: return "Discovery"
    if "create" in action or "write" in action: return "Modification"
    if "delete" in action: return "Deletion"
    if "download" in action or "clone" in action: return "Exfiltration"
    if "setiampolicy" in action: return "Privilege_Escalation"
    return "Other"

df['behavior_category'] = df['action'].apply(get_behavior_category)
df[['user', 'action', 'behavior_category']].head()

## 4. Calculate TTB Scores
Count distinct categories for each dimension per user.

In [ ]:
ttb_scores = df.groupby('user').agg(
    Terrain_Score=('terrain_category', 'nunique'),
    Time_Score=('time_context', 'nunique'),
    Behavior_Score=('behavior_category', 'nunique')
).reset_index()

print(ttb_scores)

## 5. Visualization
Plot users in 3D space to identify outliers.

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

x = ttb_scores['Terrain_Score']
y = ttb_scores['Time_Score']
z = ttb_scores['Behavior_Score']

ax.scatter(x, y, z, c='r', marker='o')

ax.set_xlabel('Terrain Diversity')
ax.set_ylabel('Time Diversity')
ax.set_zlabel('Behavior Diversity')

for i, txt in enumerate(ttb_scores['user']):
    ax.text(x[i], y[i], z[i], txt)

plt.title('TTB Analysis: User Outliers')
plt.show()